### Part 1


In [794]:
pipes = []
with open('input.txt') as f:
    lines = f.read().splitlines()
    pipes.extend(list(line) for line in lines)

In [795]:
distances = [[0 for _ in range(len(pipes[0]))] for _ in range(len(pipes))]

current = [0, 0]
for i, line in enumerate(pipes):
    for j, char in enumerate(line):
        if char == 'S':
            current = [i, j]
            break

In [796]:
def get_directions_to_move(current, pipes, previous_move=None):
    connections = []  # Connections of the current cell outwards
    # Up
    if pipes[current[0]][current[1]] in '|LJS':
        connections.append('U')
    # Down
    if pipes[current[0]][current[1]] in '|7FS':
        connections.append('D')
    # Left
    if pipes[current[0]][current[1]] in '-7JS':
        connections.append('L')
    # Right
    if pipes[current[0]][current[1]] in '-FLS':
        connections.append('R')

    # Filter self_connections by only those that point into a cell which points into the current cell
    # Up
    if 'U' in connections:
        if current[0] - 1 < 0 or pipes[current[0] - 1][current[1]] in '-LJ.':
            connections.remove('U')
    # Down
    if 'D' in connections:
        if current[0] + 1 >= len(pipes) or pipes[current[0] + 1][current[1]] in '-7F.':
            connections.remove('D')
    # Left
    if 'L' in connections:
        if current[1] - 1 < 0 or pipes[current[0]][current[1] - 1] in '|7J.':
            connections.remove('L')
    # Right
    if 'R' in connections:
        if current[1] + 1 >= len(pipes[1]) or pipes[current[0]][current[1] + 1] in '|FL.':
            connections.remove('R')

    # Filter previous_move
    if previous_move:
        if previous_move == 'U':
            connections.remove('D')
        elif previous_move == 'D':
            connections.remove('U')
        elif previous_move == 'L':
            connections.remove('R')
        elif previous_move == 'R':
            connections.remove('L')

    return connections

In [797]:
def move(current, direction):
    if direction == 'U':
        return [current[0] - 1, current[1]], direction
    elif direction == 'D':
        return [current[0] + 1, current[1]], direction
    elif direction == 'L':
        return [current[0], current[1] - 1], direction
    elif direction == 'R':
        return [current[0], current[1] + 1], direction

In [798]:
initial_directions = get_directions_to_move(current, pipes)
for initial_dir in initial_directions:
    distance = 1
    current, previous_dir = move(current, initial_dir)
    distances[current[0]][current[1]] = distance

    while True:
        directions = get_directions_to_move(current, pipes, previous_dir)
        if len(directions) > 1:
            print('JUNCTION - ERROR')
            break

        current, previous_dir = move(current, directions[0])

        if pipes[current[0]][current[1]] == 'S':
            break

        distance += 1
        if distances[current[0]][current[1]] == distance:
            print(f'FINISH - DIST = {distance}')
            break
        distances[current[0]][current[1]] = distance

FINISH - DIST = 7005


### Part 2


In [799]:
def check_cell(row, col):
    if distances[row][col] == 0 and pipes[row][col] != 'S':
        vert_count, hor_count = 0, 0

        prev_val = ''
        for r in range(row):
            if pipes[r][col] in '-LFJ7S' and distances[r][col] != 0:
                cur_val = pipes[r][col]
                if prev_val == 'F':
                    if cur_val == 'L':
                        hor_count += 1
                    elif cur_val == 'J':
                        hor_count += 2
                elif prev_val == '7':
                    if cur_val == 'J':
                        hor_count += 1
                    elif cur_val == 'L':
                        hor_count += 2
                elif cur_val in '-LFJ7S':
                    hor_count += 1
                prev_val = cur_val

        prev_val = ''
        for c in range(col):
            if pipes[row][c] in '|LFJ7S' and distances[row][c] != 0:
                cur_val = pipes[row][c]
                if prev_val == 'L':
                    if cur_val == 'J':
                        vert_count += 1
                    elif cur_val == '7':
                        vert_count += 2
                elif prev_val == 'F':
                    if cur_val == '7':
                        vert_count += 1
                    elif cur_val == 'J':
                        vert_count += 2
                elif cur_val in '|LFJ7S':
                    vert_count += 1
                prev_val = cur_val

        if vert_count % 2 and hor_count % 2:
            return True
    return False

In [800]:
import itertools

area_map = [[0 for _ in range(len(pipes[0]))] for _ in range(len(pipes))]
for r, c in itertools.product(range(len(distances)), range(len(distances[0]))):
    if a := check_cell(r, c):
        area_map[r][c] = 1

area = 0
for item in area_map:
    for c in range(len(area_map[0])):
        if item[c] == 1:
            area += 1
print(f'AREA = {area}')

AREA = 417
